In [1]:
from langchain import  PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers



In [2]:
import pinecone


In [3]:
#!pip install tf-keras
#!pip install sentence-transformers

In [4]:
# Retrieve API key and environment variables
#PINECONE_API_KEY = 'PINECONE_API_KEY', 'f376b088-cf7e-4ec6-9605-d0a8b377d7f6'
#PINECONE_API_ENV = 'PINECONE_API_ENV', 'gcp-starter'

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

extracted_data = load_pdf("../data/")


In [6]:
# we can also read the file using PyPDFLoader in a simple way
'''
from langchain.document_loaders import PyPDFLoader

# Define the path to your PDF file
pdf_file_path = "../data/your_pdf_file.pdf"

# Load the PDF using PyPDFLoader
loader = PyPDFLoader(pdf_file_path)
documents = loader.load()

# You can print or inspect the documents (contents of the PDF)
for doc in documents:
    print(doc.page_content)  # Displays the content of each page
'''


'\nfrom langchain.document_loaders import PyPDFLoader\n\n# Define the path to your PDF file\npdf_file_path = "../data/your_pdf_file.pdf"\n\n# Load the PDF using PyPDFLoader\nloader = PyPDFLoader(pdf_file_path)\ndocuments = loader.load()\n\n# You can print or inspect the documents (contents of the PDF)\nfor doc in documents:\n    print(doc.page_content)  # Displays the content of each page\n'

In [7]:
# Devide data into chunks

#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()


C:\Users\Rekha\AppData\Local\Temp\ipykernel_5960\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


### Pinecone


In [14]:
'''
#Initializing the Pinecone
#pinecone.init(api_key=PINECONE_API_KEY,  environment=PINECONE_API_ENV)


# Create an instance of the Pinecone class
pc = Pinecone(api_key=PINECONE_API_KEY)


index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

'''

'\n#Initializing the Pinecone\n#pinecone.init(api_key=PINECONE_API_KEY,  environment=PINECONE_API_ENV)\n\n\n# Create an instance of the Pinecone class\npc = Pinecone(api_key=PINECONE_API_KEY)\n\n\nindex_name="medical-chatbot"\n\n#Creating Embeddings for Each of The Text Chunks & storing\ndocsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)\n\n'

In [14]:
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone
# Create an instance of the Pinecone class

PINECONE_API_KEY = ""  

pc = Pinecone(api_key=PINECONE_API_KEY)

# Create the index instance
index_name = "medical-chatbot"  # Name of your Pinecone index
index = pc.Index(index_name)  # Create index instance using the Pinecone instance


In [15]:
index

In [16]:
# Create the Pinecone vector store from texts
from langchain.vectorstores import Pinecone as LangchainPinecone

docsearch = LangchainPinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [17]:
# Assuming embeddings is initialized correctly
docsearch_recieve = LangchainPinecone.from_existing_index(index_name=index_name,embedding=embeddings.embed_query)

C:\Users\Rekha\AppData\Roaming\Python\Python311\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [18]:
docsearch_recieve

In [19]:
# Use the vector store as a retriever
retriever = docsearch.as_retriever()

# Define your query
query = "What are Allergies"

# Retrieve relevant documents
results = retriever.get_relevant_documents(query)
print("Retrieved results:", results)

C:\Users\Rekha\AppData\Local\Temp\ipykernel_5960\1029723130.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Retrieved results: [Document(metadata={}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(metadata={}, page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nTh

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
#model_path = r"C:\models\llama-2-7b-chat.Q4_0.gguf"


In [22]:
llm=CTransformers(model="C:\models\llama-2-7b-chat.Q4_0.gguf",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
'''
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})
                          
'''

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [1]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

NameError: name 'qa' is not defined